In [1]:
# df_adjOHLCV has data only for NYSE trading days, no weekend data
# df_adjOHLCV includes data for weekends when BTC trades
# read symbols in file to list syms_in_file
# download OHLCV data for symbols in syms_in_file
# drop symbols with all NaN in OHLCV columns from df
# rename column names from ['Open', ..., 'Volume'] to ['open', ..., 'volume']
# drop weekend data by reindex to date index of index_symbol
# pickled df_adjOHLCV
# pickled df_adjOHLCV
# pickled symbols_df_adjOHLCV
# create df_symbols_close, sort df by symbols
# pickled df_symbols_close

In [2]:
# https://stackoverflow.com/questions/67690778/how-to-detect-failed-downloads-using-yfinance
import yfinance as yf
import yfinance.shared as shared
import time
import pandas as pd
# from datetime import date, timedelta, datetime
from myUtils import pickle_dump, pickle_load, read_symbols_file # NOQA
from myUtils import drop_symbols_all_NaN, chunked_list # NOQA
from myUtils import yf_download_AdjOHLCV_noAutoAdj
verbose = False  # True prints more output

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
# filename_symbols = path_data_dump + 'vg_symbols_4chars_max_clean.csv'  # symbols text file
filename_symbols = path_data_dump + 'my_symbols.csv'  # symbols text file

filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV

In [3]:
# function to adjust OHLCV
# https://github.com/ranaroussi/yfinance/issues/687
def auto_adjust(data):
    df = data.copy()
    ratio = df["Close"] / df["Adj Close"]
    df["Adj Open"] = df["Open"] / ratio
    df["Adj High"] = df["High"] / ratio
    df["Adj Low"] = df["Low"] / ratio

    df.drop(
        ["Open", "High", "Low", "Close"],
        axis=1, inplace=True)

    df.rename(columns={
        "Adj Open": "Open", "Adj High": "High",
        "Adj Low": "Low", "Adj Close": "Close"
    }, inplace=True)

    # df = df[["Open", "High", "Low", "Close", "Volume"]]
    return df[["Open", "High", "Low", "Close", "Volume"]]

def close_vs_Yahoo (symbols, df):
  for symbol in symbols:
    s = df.iloc[-250]
    
    sDate = s.name.strftime('%Y-%m-%d')
    sClose = s[symbol].Close

    df_sym = yf.Ticker(symbol).history(period='2y')
    # 10:45PM got Adj Close
    yhClose = df_sym.loc[sDate]['Close']

    abs_pct_diff = abs(1 - sClose/yhClose)*100
    print(f'symbol:  {symbol:>4}   Date: {sDate:13}df_Close: {sClose:>10.5f} \
    Yahoo_Close: {yhClose:>10.5f}   %_dif_Close: {abs_pct_diff:>7.5f}')
    if abs_pct_diff > .0001:
      msg_err = f'{symbol}  %_dif_Close > .0001'
      # raise SystemExit(msg_err)
      print(msg_err)  # even if discrepancy, the relative relationship of OHLCV in df_close matches Yahoo_Close
    if symbol == symbols[-1]:
      msg_done = f"No errors found.  Symbols' 'Adjusted Close' matched Yahoo symbols' Close "
      print(msg_done)


In [4]:
# Stop if last date in df_adjOHLCV is the same as Yahoo download
index_symbol = "XOM"  
df_XOM = yf.download(index_symbol)
download_last_date = df_XOM.index[-1].strftime('%Y-%m-%d')
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
print(f'download_last_date: {download_last_date}')
df = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)
df_adjOHLCV_last_date = df.index[-1].strftime('%Y-%m-%d')
print(f'df_adjOHLCV_last_date: {df_adjOHLCV_last_date}')

##########  Allow Repeated Download on the Same Day  ###########################################
# if download_last_date == df_adjOHLCV_last_date:
#   msg_stop = f"df_adjOHLCV is up-to-date,\nlast date from df_adjOHLCV is {df_adjOHLCV_last_date},\nlast date from Yahoo's {index_symbol} download is {download_last_date}"
#   raise SystemExit(msg_stop )
######################################################

[*********************100%***********************]  1 of 1 completed
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV
download_last_date: 2023-07-14
df_adjOHLCV_last_date: 2023-07-13


In [5]:
# verify df test_symbols' close against Yahoo
# even if discrepancy, the relative relationship of OHLCV in df_close matches Yahoo_Close
test_symbols = ['A', 'LMT', 'SPY']
_df_list=[]
_df = yf_download_AdjOHLCV_noAutoAdj(test_symbols, verbose=False)
_df_list.append(_df)
_df = pd.concat(_df_list, axis=1)
# get unique symbols from column name
_l_symbols = list(set(_df.columns.get_level_values(0)))
_l_symbols.sort()

_df_adj_list=[]
# adjust OHLC using 'Adj Close'
for symbol in _l_symbols:
  _df1 = auto_adjust(_df[symbol])
  _df_adj_list.append(_df1)

_df_adj = pd.concat(_df_adj_list, axis=1)
_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
# create multilevel column names
_col_names = pd.MultiIndex.from_product([_l_symbols, _cols])
_df_adj.columns = _col_names

close_vs_Yahoo(test_symbols, _df_adj)

[*********************100%***********************]  3 of 3 completed
symbol:     A   Date: 2022-07-18   df_Close:  114.33505     Yahoo_Close:  114.33504   %_dif_Close: 0.00001
symbol:   LMT   Date: 2022-07-18   df_Close:  377.40826     Yahoo_Close:  377.40826   %_dif_Close: 0.00000
symbol:   SPY   Date: 2022-07-18   df_Close:  375.81308     Yahoo_Close:  375.81308   %_dif_Close: 0.00000
No errors found.  Symbols' 'Adjusted Close' matched Yahoo symbols' Close 


In [6]:
# read symbols in file to a list and convert to chunks
#  each chunk must have more than 1 symbol, otherwise, symbol won't appear as column name
symbols_in_file = read_symbols_file(filename_symbols)
print(f'symbols in file: {len(symbols_in_file)}')
symbols_add = ['SPY', 'VGT', 'FTEC', 'QQQ', 'VWO', 'VTV', 'BNDX', 'USO', 'JJC', 'BCI', 'NG', 'GBTC', 'BTC-USD', 'ETH-USD', 'SGOV', 'GLD']
print(f'symbols added:   {len(symbols_add)}')
symbols = symbols_in_file + symbols_add
# https://stackoverflow.com/questions/12897374/get-unique-values-from-a-list-in-python
symbols = list(set(symbols))  # keep unique symbols
symbols.sort()  # sort unique symbols in alphabetical order
print(f'unique symbols:  {len(symbols)}')
symbols_chunks = chunked_list(symbols, 400)  # e.g. [['A', 'BB', ...], ['CC', 'DD', ...], ..., ['Z', 'ZWS', ...]]
for chunk in symbols_chunks:
  if len(chunk) == 1:
    raise SystemExit(f'ERROR, Must be more than 1 symbol in each chunk\nsymbols in chunk: {chunk}')

symbols in file: 1578
symbols added:   16
unique symbols:  1587


In [7]:
df_list=[]
symbols_download_err = []
# took 24 minutes to download 1917 symbols without error caused by Yahoo
# for i, symbols in enumerate(symbols_chunks):
for i, symbols in enumerate(symbols_chunks):
  df = yf_download_AdjOHLCV_noAutoAdj(symbols, verbose=False)  
  symbols_download_err.append(list(shared._ERRORS.keys()))
  # print(f'symbols_download_err: {symbols_download_err}')
  print(f'\nsymbols_download_err: {symbols_download_err}')  
  df_list.append(df)
  # pause between download
  if i < len(symbols_chunks) - 1 :  # skip pause after last download
    print(f'downloaded symbols from chuck {i}, sleep start')
    # sleep 78(18m 25s)
    time.sleep(78)
    print(f'downloaded symbols from chuck {i}, sleep ends')
  else:
    print(f'downloaded symbols from all chucks')

print(f'symbols_download_err: {symbols_download_err}')

[*********************100%***********************]  400 of 400 completed

2 Failed downloads:
- AAWW: No data found, symbol may be delisted
- ATCO: No data found, symbol may be delisted

symbols_download_err: [['AAWW', 'ATCO']]
downloaded symbols from chuck 0, sleep start
downloaded symbols from chuck 0, sleep ends
[*********************100%***********************]  400 of 400 completed

2 Failed downloads:
- FRC: No data found, symbol may be delisted
- IAA: No data found, symbol may be delisted

symbols_download_err: [['AAWW', 'ATCO'], ['FRC', 'IAA']]
downloaded symbols from chuck 1, sleep start
downloaded symbols from chuck 1, sleep ends
[*********************100%***********************]  400 of 400 completed

symbols_download_err: [['AAWW', 'ATCO'], ['FRC', 'IAA'], []]
downloaded symbols from chuck 2, sleep start
downloaded symbols from chuck 2, sleep ends
[*********************100%***********************]  387 of 387 completed

2 Failed downloads:
- SJR: No data found, symbol may b

In [8]:
# flatten symbols_download_err which is a list-of-lists
l_symbols_err = [val for sublist in symbols_download_err for val in sublist]
pickle_dump(l_symbols_err, path_data_dump, 'l_symbols_err')
l_symbols_err

['AAWW', 'ATCO', 'FRC', 'IAA', 'SJR', 'TA']

In [9]:
# if l_symbols_err:  # re-download symbols with download error 
#   symbols_download_err = []
#   # took 24 minutes to download 1917 symbols without error caused by Yahoo
#   # for i, symbols in enumerate(symbols_chunks):
#   for i, symbols in enumerate(l_symbols_err):
#     df = yf_download_AdjOHLCV_noAutoAdj(symbols, verbose=False)  
#     symbols_download_err.append(list(shared._ERRORS.keys()))
#     # print(f'symbols_download_err: {symbols_download_err}')
#     print(f'\nsymbols_download_err: {symbols_download_err}')  
#     df_list.append(df)
#     # pause between download
#     if i < len(symbols_chunks) - 1 :  # skip pause after last download
#       print(f'downloaded symbols from chuck {i}, sleep start')
#       # sleep 78(18m 25s)
#       time.sleep(78)
#       print(f'downloaded symbols from chuck {i}, sleep ends')
#     else:
#       print(f'downloaded symbols from all chucks')

#   print(f'symbols_download_err: {symbols_download_err}')

In [10]:
df = pd.concat(df_list, axis=1)
if l_symbols_err:  # if list is not empty, drop symbols with errors
  print(f'l_symbols_err is not empty: {l_symbols_err}')  
  df.drop(l_symbols_err, axis=1, level=0, inplace=True)  # drop symbols with errors

l_symbols = list(set(df.columns.get_level_values(0)))  # get unique symbols
l_symbols.sort()
df = df[l_symbols]  # sort columns by list
print(f"Full path to pickled df_OHLCVA_downloaded:  {path_data_dump}{filename_pickled_df_OHLCVA_downloaded}")
pickle_dump(df, path_data_dump, filename_pickled_df_OHLCVA_downloaded)  # save OHLCA data

l_symbols_err is not empty: ['AAWW', 'ATCO', 'FRC', 'IAA', 'SJR', 'TA']
Full path to pickled df_OHLCVA_downloaded:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCVA_downloaded 


In [11]:
# adjust OHLC
df_adj_list=[]
for symbol in l_symbols:
  _df = auto_adjust(df[symbol])
  df_adj_list.append(_df)

df_adj = pd.concat(df_adj_list, axis=1)
cols = ['Open', 'High', 'Low', 'Close', 'Volume']
col_names = pd.MultiIndex.from_product([l_symbols, cols])  # create multilevel column names
df_adj.columns = col_names  # reindex columns

print(f"Full path to pickled df_adjOHLCV: {path_data_dump}{filename_pickled_df_adjOHLCV}")
pickle_dump(df_adj, path_data_dump, filename_pickled_df_adjOHLCV)  # save adjusted OHLCV data

Full path to pickled df_adjOHLCV: C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV


In [12]:
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV)

In [13]:
# drop symbols with all NaN in OHLCV columns from df
df_adjOHLCV, symbols_OHLCV, symbols_dropped = drop_symbols_all_NaN(df_adjOHLCV, verbose)
print(f'symbols with all NaN dropped from df_adjOHLCV: {symbols_dropped}')

Symbol's OHLCV are all NaN: []
symbols with all NaN dropped from df_adjOHLCV: []


In [14]:
# rename columns OHLCV *ONLY AFTER* dropping symbols with all NaN from df,
#   symbols with all NaN has an added AdjClose column and will cause errors  
#  rename column names from ['Open', ..., 'Volume'] to ['open', ..., 'volume']
#  .remove_unused_levels() prevents ValueError
#   e.g ValueError: On level 1, code max (5) >= length of level (5). NOTE: this index is in an inconsistent state
# The error may be caused by removing symbols from the dataframe with all NaN in OHLCV columns
df_adjOHLCV.columns = df_adjOHLCV.columns.remove_unused_levels()
# # set_levels reorders df columns in alphabetical order, so the list of column names also needs to be in alphabetical order
# df_adjOHLCV.columns = df_adjOHLCV.columns.set_levels(['close', 'high', 'low', 'open', 'volume'], level=1)

In [15]:
# drop weekend data by re-indexing to date-index of index_symbol
myNaN = float('nan')
# use Exxon's date as proxy for NYSE trading dates
df_adjOHLCV = df_adjOHLCV.reindex(df_XOM.index, fill_value=myNaN)

In [16]:
# pickle df_adjOHLCV and symbols
# print(f"Full path to pickled df_adjOHLCV_downloaded:  {path_data_dump}{filename_pickled_df_adjOHLCV_downloaded}")
# pickle_dump(df_adjOHLCV_downloaded, path_data_dump, filename_pickled_df_adjOHLCV_downloaded, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
pickle_dump(df_adjOHLCV, path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)
print(f"Full path to pickled symbols_df_adjOHLCV:  {path_data_dump}{filename_pickled_symbols_df_adjOHLCV}")
pickle_dump(symbols_OHLCV, path_data_dump, filename_pickled_symbols_df_adjOHLCV, verbose=verbose)

Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV
Full path to pickled symbols_df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/symbols_df_adjOHLCV


In [17]:
symbols_OHLCV = list(set([i[0] for i in list(df_adjOHLCV)]))
df_symbols_close = df_adjOHLCV.xs('Close', level=1, axis=1)
pickle_dump(df_symbols_close, path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)

In [18]:
# retrieve pickled files

print(f"Full path to pickled df_OHLCVA_downloaded:  {path_data_dump}{filename_pickled_df_OHLCVA_downloaded}")
df_OHLCVA_downloaded = pickle_load(path_data_dump, filename_pickled_df_OHLCVA_downloaded, verbose=verbose)

print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)

print(f"Full path to pickled symbols_df_adjOHLCV:  {path_data_dump}{filename_pickled_symbols_df_adjOHLCV}")
symbols_df = pickle_load(path_data_dump, filename_pickled_symbols_df_adjOHLCV, verbose=verbose)

Full path to pickled df_OHLCVA_downloaded:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCVA_downloaded 
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV
Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled symbols_df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/symbols_df_adjOHLCV


In [19]:
print(f'df_OHLCVA_downloaded.info():\n{df_OHLCVA_downloaded.info()}\n')
print(f'df_adjOHLCV.info():\n{df_adjOHLCV.info()}\n')
print(f'df_close.info():\n{df_close.info()}\n')
print(f'len(symbols_df):\n{len(symbols_df)}\n')

<class 'pandas.core.frame.DataFrame'>
Index: 16492 entries, 1962-01-02 00:00:00 to 2023-07-16 00:00:00
Columns: 9486 entries, ('A', 'Open') to ('ZWS', 'Volume')
dtypes: float64(9486)
memory usage: 1.2+ GB
df_OHLCVA_downloaded.info():
None

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15489 entries, 1962-01-02 to 2023-07-14
Columns: 7905 entries, ('A', 'Open') to ('ZWS', 'Volume')
dtypes: float64(7905)
memory usage: 934.3 MB
df_adjOHLCV.info():
None

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15489 entries, 1962-01-02 to 2023-07-14
Columns: 1581 entries, A to ZWS
dtypes: float64(1581)
memory usage: 186.9 MB
df_close.info():
None

len(symbols_df):
1581



In [20]:
count_symbols = len(symbols_df)
count_df_close_cols = len(df_close.columns)
count_cols_df_OHLCVA_downloaded = len(df_OHLCVA_downloaded.columns)
count_cols_df_adjOHLCV = len(df_adjOHLCV.columns)
if count_symbols != count_df_close_cols:
  raise SystemExit(f'symbol count: {count_symbols} does not equal df_close column count: {count_df_close_cols}')
if count_symbols * 6 != count_cols_df_OHLCVA_downloaded:
  raise SystemExit(f'6 x symbol count: {6 * count_symbols} does not equal df_OHLCVA_downloaded column count: {count_cols_df_OHLCVA_downloaded}')
if count_symbols * 5 != count_cols_df_adjOHLCV:
  raise SystemExit(f'5 x symbol count: {5 * count_symbols} does not equal df_adjOHLCV column count: {count_cols_df_adjOHLCV}')
print(f'No error found, Column counts in df_close, df_OHLCVA, df_adjOHLCV matched symbol count')


No error found, Column counts in df_close, df_OHLCVA, df_adjOHLCV matched symbol count


In [21]:
print(f'count_symbols: {count_symbols}')
print(f'count_df_close_cols: {count_df_close_cols}')
print(f'count_cols_df_OHLCVA_downloaded: {count_cols_df_OHLCVA_downloaded}')
print(f'count_cols_df_adjOHLCV: {count_cols_df_adjOHLCV}')

count_symbols: 1581
count_df_close_cols: 1581
count_cols_df_OHLCVA_downloaded: 9486
count_cols_df_adjOHLCV: 7905


In [22]:
close_vs_Yahoo(test_symbols, df_adjOHLCV)

symbol:     A   Date: 2022-07-18   df_Close:  114.33505     Yahoo_Close:  114.33505   %_dif_Close: 0.00000
symbol:   LMT   Date: 2022-07-18   df_Close:  377.40826     Yahoo_Close:  377.40826   %_dif_Close: 0.00000
symbol:   SPY   Date: 2022-07-18   df_Close:  375.81308     Yahoo_Close:  375.81305   %_dif_Close: 0.00001
No errors found.  Symbols' 'Adjusted Close' matched Yahoo symbols' Close 


In [23]:
_sym = 'NOC'
s_start, s_end = -252, -248
df_adjOHLCV[_sym].iloc[s_start:s_end]
df_OHLCVA_downloaded[_sym].iloc[s_start:s_end]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-11-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-07 00:00:00,523.049988,537.580017,520.559998,534.770020,528.771912,804500.0
2022-11-08 00:00:00,533.340027,542.140015,532.330017,540.929993,534.862793,652600.0
2022-11-09 00:00:00,540.659973,543.190002,532.809998,533.140015,527.160217,601600.0


In [24]:
df_adjOHLCV.tail()

A                                                 \
                  Open        High         Low       Close     Volume   
Date                                                                    
2023-07-10  116.730003  119.519997  116.559998  118.360001  1889200.0   
2023-07-11  118.300003  118.980003  117.639999  118.480003  1096700.0   
2023-07-12  120.000000  120.489998  119.150002  120.000000  1595400.0   
2023-07-13  120.760002  120.870003  118.989998  119.349998  1229800.0   
2023-07-14  119.540001  120.389999  118.250000  119.750000  1225400.0   

                   AA                                              ...  \
                 Open       High        Low      Close     Volume  ...   
Date                                                               ...   
2023-07-10  33.700001  34.570000  33.599998  34.139999  3891400.0  ...   
2023-07-11  33.599998  34.869999  33.330002  34.689999  5218400.0  ...   
2023-07-12  35.810001  35.950001  35.000000  35.049999  4288700.0  ...   
2023-07-13  35.700001  37.169998  35.549999  36.689999  5835900.0  ...   
2023-07-14  35.500000  35.520000  34.049999  34.520000  8079500.0  ...   

                 ZUMZ                                                   ZWS  \
                 Open       High        Low      Close    Volume       Open   
Date                                                                          
2023-07-10  17.370001  17.610001  16.809999  17.309999  362400.0  26.049999   
2023-07-11  17.450001  17.799999  17.379999  17.730000  278900.0  27.020000   
2023-07-12  18.059999  18.420000  17.680000  17.900000  296700.0  27.480000   
2023-07-13  17.910000  17.910000  17.450001  17.459999  307100.0  27.450001   
2023-07-14  17.420000  17.670000  17.010000  17.650000  376900.0  27.209999   

                                                        
                 High        Low      Close     Volume  
Date                                                    
2023-07-10  26.730000  26.049999  26.690001  1125300.0  
2023-07-11  27.209999  26.780001  27.100000   881300.0  
2023-07-12  27.780001  27.090000  27.180000   880000.0  
2023-07-13  27.639999  27.200001  27.379999   777300.0  
2023-07-14  27.430000  26.920000  27.160000   460600.0  

[5 rows x 7905 columns]

In [25]:
syms_def = ['NOC', 'HII', 'AVAV', 'LMT', 'CW', 'HEI']
_df = df_close[syms_def]
_df.tail()


,NOC,HII,AVAV,LMT,CW,HEI
Date,,,,,,
2023-07-10,452.589996,227.869995,97.970001,460.380005,185.479996,174.990005
2023-07-11,460.750000,230.970001,99.669998,466.290009,187.240005,176.440002
2023-07-12,453.350006,229.589996,99.889999,461.959991,190.789993,174.830002
2023-07-13,457.029999,228.279999,99.910004,466.380005,188.259995,174.059998
2023-07-14,450.640015,228.350006,96.449997,464.309998,188.259995,170.490005


In [26]:
syms_LNG = ['LNG', 'CHK', 'GLNG']
_df = df_close[syms_LNG]
_df.tail()

,LNG,CHK,GLNG
Date,,,
2023-07-10,152.830002,82.839996,22.469999
2023-07-11,158.380005,84.230003,22.900000
2023-07-12,156.979996,84.110001,23.510000
2023-07-13,157.350006,83.400002,23.610001
2023-07-14,153.899994,81.120003,23.450001


In [27]:
_df = df_close[symbols_add]
_df.tail()

,SPY,VGT,FTEC,QQQ,VWO,VTV,BNDX,USO,JJC,BCI,NG,GBTC,BTC-USD,ETH-USD,SGOV,GLD
Date,,,,,,,,,,,,,,,,
2023-07-10,439.660004,437.230011,129.250000,366.359985,40.590000,141.259995,48.200001,65.540001,19.559999,20.379999,4.11,19.990000,30414.470703,1880.556396,100.330002,178.779999
2023-07-11,442.459991,438.880005,129.690002,368.170013,40.950001,142.619995,48.230000,67.029999,19.559999,20.580000,4.24,19.930000,30620.951172,1878.336060,100.339996,179.449997
2023-07-12,446.019989,443.470001,131.039993,372.820007,41.730000,142.979996,48.450001,67.870003,19.559999,20.629999,4.38,19.700001,30391.646484,1872.113770,100.360001,181.880005
2023-07-13,449.559998,450.079987,132.970001,379.149994,42.189999,143.289993,48.720001,69.089996,19.559999,20.889999,4.44,21.379999,31476.048828,2006.511475,100.400002,182.009995
2023-07-14,449.279999,448.470001,132.529999,379.070007,41.930000,142.830002,48.610001,67.470001,19.559999,20.850000,4.31,19.750000,30334.068359,1939.347046,100.410004,181.429993


In [28]:
# syms_def = ['NOC', 'HII', 'AVAV', 'LMT', 'CW', 'HEI']
s_start, s_end = -252, -247
_df = df_close[test_symbols].iloc[s_start:s_end]
_df

,A,LMT,SPY
Date,,,
2022-07-14,115.099869,390.106110,371.837952
2022-07-15,118.030006,388.225311,378.941956
2022-07-18,114.335052,377.408264,375.813080
2022-07-19,118.258453,380.429230,385.967194
2022-07-20,119.053070,385.145844,388.427063
